# 1.  Load libraries

In [3]:
# Data Manipulation Libraries
import os
import pandas as pd
import numpy as np

# Plotting Libraries
from matplotlib import pyplot as plt
import seaborn as sns

# Machine Learning Libraries
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# 2. load data

In [4]:
for dirname, _, filenames in os.walk('../data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



../data/sample_submission.csv
../data/store.csv
../data/test.csv
../data/train.csv


In [15]:
df_sample=pd.read_csv('../data/sample_submission.csv', low_memory=False)
df_store=pd.read_csv('../data/store.csv', low_memory=False)
df_test=pd.read_csv('../data/test.csv', low_memory=False)
df_train=pd.read_csv('../data/train.csv', low_memory=False)

# test data

In [25]:
print(f"df_train shape: {df_train.shape}")
print(f"df_test shape: {df_test.shape}")
print(f"df_store shape: {df_store.shape}")

df_train shape: (1017209, 9)
df_test shape: (41088, 8)
df_store shape: (1115, 10)


In [32]:
print( "df_train shape:", df_train.info())
print("df_test shape:", df_test.info())
print( "df_store shape:", df_store.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   Store          1017209 non-null  int64 
 1   DayOfWeek      1017209 non-null  int64 
 2   Date           1017209 non-null  object
 3   Sales          1017209 non-null  int64 
 4   Customers      1017209 non-null  int64 
 5   Open           1017209 non-null  int64 
 6   Promo          1017209 non-null  int64 
 7   StateHoliday   1017209 non-null  object
 8   SchoolHoliday  1017209 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 69.8+ MB
df_train shape: None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41088 entries, 0 to 41087
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             41088 non-null  int64  
 1   Store          41088 non-null  int64  
 2   DayOfWeek      41088 non-null  i

# Check Missing Values

In [33]:
# Check for missing values in each dataset
print("Missing values in df_train:")
print(df_train.isnull().sum())

print("Missing values in df_test:")
print(df_test.isnull().sum())

print("Missing values in df_store:")
print(df_store.isnull().sum())


Missing values in df_train:
Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64
Missing values in df_test:
Id                0
Store             0
DayOfWeek         0
Date              0
Open             11
Promo             0
StateHoliday      0
SchoolHoliday     0
dtype: int64
Missing values in df_store:
Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            3
CompetitionOpenSinceMonth    354
CompetitionOpenSinceYear     354
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64


# Handle Missing Values

In [35]:
# Fill missing 'Open' values in df_test with 1 (assuming store was open)
df_test['Open'] = df_test['Open'].fillna(1)

# Fill missing 'CompetitionDistance' with the median value
df_store['CompetitionDistance'] = df_store['CompetitionDistance'].fillna(df_store['CompetitionDistance'].median())

# Fill missing 'CompetitionOpenSinceMonth' and 'CompetitionOpenSinceYear' with 0
df_store['CompetitionOpenSinceMonth'] = df_store['CompetitionOpenSinceMonth'].fillna(0)
df_store['CompetitionOpenSinceYear'] = df_store['CompetitionOpenSinceYear'].fillna(0)

# Fill missing 'Promo2SinceWeek' and 'Promo2SinceYear' with 0
df_store['Promo2SinceWeek'] = df_store['Promo2SinceWeek'].fillna(0)
df_store['Promo2SinceYear'] = df_store['Promo2SinceYear'].fillna(0)

# Fill missing 'PromoInterval' with 'No Promo'
df_store['PromoInterval'] = df_store['PromoInterval'].fillna('No Promo')


# Verify Missing Values Have Been Handled

In [36]:
# Verify that no missing values remain
print("Missing values in df_test after handling:")
print(df_test.isnull().sum())

print("Missing values in df_store after handling:")
print(df_store.isnull().sum())


Missing values in df_test after handling:
Id               0
Store            0
DayOfWeek        0
Date             0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64
Missing values in df_store after handling:
Store                        0
StoreType                    0
Assortment                   0
CompetitionDistance          0
CompetitionOpenSinceMonth    0
CompetitionOpenSinceYear     0
Promo2                       0
Promo2SinceWeek              0
Promo2SinceYear              0
PromoInterval                0
dtype: int64


# Feature Engineering
## 1. Convert Date Columns to Datetime:

In [37]:
df_train['Date'] = pd.to_datetime(df_train['Date'])
df_test['Date'] = pd.to_datetime(df_test['Date'])


## 2.Extract Features from Date

In [38]:
# For training data
df_train['Year'] = df_train['Date'].dt.year
df_train['Month'] = df_train['Date'].dt.month
df_train['Day'] = df_train['Date'].dt.day
df_train['WeekOfYear'] = df_train['Date'].dt.isocalendar().week

# For test data
df_test['Year'] = df_test['Date'].dt.year
df_test['Month'] = df_test['Date'].dt.month
df_test['Day'] = df_test['Date'].dt.day
df_test['WeekOfYear'] = df_test['Date'].dt.isocalendar().week


## 3. Merge Datasets:

In [40]:
# Merge training data with store data
df_train = pd.merge(df_train, df_store, on='Store', how='left')

# Merge test data with store data
df_test = pd.merge(df_test, df_store, on='Store', how='left')


In [41]:
print("df_train head after merging:")
print(df_train.head())

print("df_test head after merging:")
print(df_test.head())


df_train head after merging:
   Store  DayOfWeek       Date  Sales  Customers  Open  Promo StateHoliday  \
0      1          5 2015-07-31   5263        555     1      1            0   
1      2          5 2015-07-31   6064        625     1      1            0   
2      3          5 2015-07-31   8314        821     1      1            0   
3      4          5 2015-07-31  13995       1498     1      1            0   
4      5          5 2015-07-31   4822        559     1      1            0   

   SchoolHoliday  Year  ...  WeekOfYear  StoreType  Assortment  \
0              1  2015  ...          31          c           a   
1              1  2015  ...          31          a           a   
2              1  2015  ...          31          a           a   
3              1  2015  ...          31          c           c   
4              1  2015  ...          31          a           a   

  CompetitionDistance CompetitionOpenSinceMonth  CompetitionOpenSinceYear  \
0              1270.0       

In [42]:
print("Duplicates in df_train:", df_train.duplicated().sum())
print("Duplicates in df_test:", df_test.duplicated().sum())


Duplicates in df_train: 0
Duplicates in df_test: 0


In [43]:
df_train.to_csv('../data/processed_train.csv', index=False)
df_test.to_csv('../data/processed_test.csv', index=False)
